In [99]:
import json
import glob
import cv2
import os
from tqdm import tqdm
from PIL import ImageFile

In [33]:
classes = ['Ch01','Ch02','Ch03','Ch04','Ch05','Ch06']

In [100]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [18]:
mother_dir = 'Training/label/'
for c in classes:
    label_dir = mother_dir + c
    label_json = glob.glob(label_dir + '/*.json')
    for label in tqdm(label_json):
        with open(label, 'r', encoding='utf-8') as j:
            data = json.load(j)
            status = data['metadata']['Disease']
            init_img_dir = 'Training/raw/' + data['fileName'][:-4] + 'jpg'
            img_dir = 'Training/raw/'+ c + '/'+ status + '/' + data['fileName'][:-4] + 'jpg'
            try:
                os.rename(init_img_dir, img_dir)
            except:
                continue
            

100%|██████████| 22542/22542 [00:02<00:00, 8111.77it/s]


In [114]:
import numpy as np
import matplotlib.pyplot as plt
def padding(json_path):
    with open(json_path, 'r', encoding='utf-8') as j:
        data = json.load(j)
        # 사진 경로
        status = data['metadata']['Disease']
        c = data['metadata']['Disease-Name']
        img_dir = 'Training/raw/'+ c + '/'+ status + '/' + data['fileName'][:-4] + 'jpg'
        img = Image.open(img_dir)
        img = img.convert('RGB')
        img_array = np.array(img)
        w, h = img.size
        if w == 224 and h == 224:
            return

        min_x, min_y = 1e9, 1e9
        max_x, max_y = -1e9, -1e9
        
        # ROI 추출
        points = data['annotations'][0]['points']
        for point in points:
            x, y = point
            min_x = min(min_x,x)
            min_y = min(min_y,y)
            max_x = max(max_x,x)
            max_y = max(max_y,y)
        dx, dy = max_x-min_x, max_y-min_y
        w = max(dx,dy)
    

    # Determine the size of the padding to add
    right_pad = 4100 - w
    bottom_pad = 4100 - h

    # Create a padding array with zeros
    pad_width = ((0, bottom_pad), (0, right_pad), (0, 0))
    padded_img_array = np.pad(img_array, pad_width, mode='constant', constant_values=0)

    # Convert the padded array back to an image and save
    # padded_img = Image.fromarray(padded_img_array)

    if min_x >= 100 and min_y >= 100:    
        cropped_img = padded_img_array[min_y-100:min_y+w+100,min_x-100:min_x+w+100]
    else:
        min_margin = min(min_x,min_y)
        cropped_img = padded_img_array[min_y-min_margin:min_y+w+100,min_x-min_margin:min_x+w+100]

    
    
    cropped_img = cv2.resize(cropped_img,(224,224))
    cv2.imwrite(img_dir, cropped_img)
    return



In [115]:
error_array = []
for c in classes[-1:]:
    print(c)
    label_dir = mother_dir + c
    label_json = glob.glob(label_dir + '/*.json')
    # print(label_json[:1])
    for label in tqdm(label_json):
        try:
            padding(label)
        except TypeError:
            error_array.append(label)
            print('error')
            continue


Ch06


 21%|██        | 4740/22542 [00:03<00:17, 989.68it/s] 

error


 35%|███▍      | 7807/22542 [00:13<00:50, 289.88it/s]

error


 36%|███▋      | 8226/22542 [00:21<12:43, 18.75it/s] 

In [55]:
# 패딩을 위한 이미지 데이터 크기 최대값 (Lateral)
from PIL import Image

for c in classes:
    print(c)
    label_dir = mother_dir + c
    label_json = glob.glob(label_dir + '/*.json')
    # print(label_json[:1])
    # min_x,min_y,max_x,max_y = 1e9,1e9,-1e9,-1e9
    max_size = 0
    for label in label_json:
        with open(label, 'r', encoding='utf-8') as j:
            data = json.load(j)
            # 사진 경로
            pose    = data['metadata']['Position']
            if pose != 'Lateral':
                status  = data['metadata']['Disease']
                img_dir = 'Training/raw/'+ c + '/'+ status + '/' + data['fileName'][:-4] + 'jpg'
                img = Image.open(img_dir)
                w, h = img.size
                max_size = max(max_size, max(w, h))
    print(max_size)

Ch01
0
Ch02
0
Ch03
4000
Ch04
4000
Ch05
3947
Ch06
4000


모든 사진의 최대 크기를 4000이라 가정해도 무방하다. 대부분 3990정도

In [ ]:
lateral_max_size = {'Ch01':3986,'Ch02':3991,'Ch03':0,'Ch04':3995,'Ch05':3983,'Ch06':3995}
VD_max_size = {'Ch01':0,'Ch02':0,'Ch03':4000,'Ch04':4000,'Ch05':4000,'Ch06':4000}